In [13]:
import networkx as nx
import matplotlib.pyplot as plt
import heapq
from collections import deque
from IPython.display import clear_output

In [14]:
#The class for showing the graph
class GraphVisualization:
    def __init__(self, title):
        self.G = nx.Graph()
        self.title = title

    def add_vertex(self, vertex):
        self.G.add_node(vertex)
        
    def add_edge(self, edge1, edge2):
        self.G.add_edge(edge1, edge2)
        
    def remove_vertex(self, vertex):
        self.G.remove_node(vertex)
        
    def remove_edge(self, edge1, edge2):
        self.G.remove_edge(edge1, edge2)
    
    def display(self, layout):
        plt.title(self.title)
        pos = layout(self.G)  # Define the layout for the nodes
        nx.draw_networkx_nodes(self.G, pos, node_color='lightblue', node_size=750)  # Draw the nodes
        nx.draw_networkx_edges(self.G, pos, edge_color='gray')  # Draw the edges
        nx.draw_networkx_labels(self.G, pos, font_color='black')  # Draw the node labels
        plt.axis('off')  # Turn off the axis
        plt.show()

In [15]:
class Member:
    def __init__(self, name, age, location, friends=None):
        self.name = name
        self.age = age
        self.location = location
        self.friends = friends if friends else []

In [16]:
class Graph:
    def __init__(self, title):
        # Map members' names to the Member object
        self.members = {} 
        self.visualizer = GraphVisualization(title)
        
    def __str__(self):
        return str([member.name for member in self.members.values()])
    
    def add_member(self, name, age, location):
        'Add member to the graph '
        # Check if the object with this name exists
        if name not in self.members:
            self.members[name] = Member(name, age, location)
            self.visualizer.add_vertex(name)
        else:
            print("The member already exists")
            
    def add_relationship(self, name1, name2):
        'Add relationship to the graph'
        if name1 in self.members and name2 in self.members:
            self.members[name1].friends.append(self.members[name2])
            self.members[name2].friends.append(self.members[name1])
            self.visualizer.add_edge(name1, name2)
        else:
            print("The members have not existed.")
            
    def remove_member(self, name):
        'Remove member from the graph'
        if name in self.members:
            for friend in self.members[name].friends:
                friend.friends.remove(self.members[name])
                
            del self.members[name]
            self.visualizer.remove_vertex(name)
        else:
            print("This member does not exist.")
            
            
    def remove_relationship(self, name1, name2):
        'Remove relationship from the graph'
        if name1 in self.members and name2 in self.members:
            if self.members[name2] in self.members[name1].friends and self.members[name1] in self.members[name2].friends:
                self.members[name1].friends.remove(self.members[name2])
                self.members[name2].friends.remove(self.members[name1])
                self.visualizer.remove_edge(name1, name2)
            else:
                print("This relationship does not exist.")
                
        else:
            print("One of the members does not exiet.")
            
    def find_friends(self, name):
        'Find the friends of a member'
        if name in self.members:
            return [friend.name for friend in self.members[name].friends]
        else:
            print("The member has not existed. ")
            return []
    
    # Dijkstra's algorithm
    def shortest_path(self, start, end):
        'Find the shortest path between members'
        if (start not in self.members or
            end not in self.members):
            return -1

        visited = set()
        pq = [(0, start)]  # Push tuple of (weight, member_name)

        while pq:
            w, member_name = heapq.heappop(pq)  
            if member_name == end:
                return w

            visited.add(member_name)
            member = self.members[member_name]

            for friend in member.friends:
                if friend.name not in visited:
                    heapq.heappush(pq, (w + 1, friend.name)) # Push tuple

        return -1
    
    def shortest_path_dfs(self, start, end):
        'Find the shortest path between members using DFS'
        if (start not in self.members or
            end not in self.members):
            return -1
        
        visited = set()
        
        def dfs(member_name):
            if member_name == end:
                return len(visited)
            
            visited.add(member_name)

            mn = float('inf')
            for friend in self.members[member_name].friends:
                if friend.name not in visited:
                    mn = min(mn, dfs(friend.name))
                
            visited.remove(member_name)
            
            return mn
        
        return dfs(start)
        
    def find_k_new_friends(self, member, k):
        'Find k new friends for a member'
        if member not in self.members:
            return []
        
        q = deque([friend.name for friend in self.members[member].friends])
        visited = set([member])
        new_friends = []
        
        while q:
            for i in range(len(q)):
                if len(new_friends) == k:
                    return new_friends
                
                member_name = q.popleft()
                visited.add(member_name)
                
                for friend in self.members[member_name].friends:
                    if friend.name not in visited:
                        new_friends.append(friend.name)
                        q.append(friend.name)
                        
        return new_friends

    def display_graph(self, layout=nx.spring_layout):
        'Display the graph'
        self.visualizer.display(layout)

In [17]:
class GraphCLIApp(Graph):
    def __init__(self):
        super().__init__('Social Network Graph')
        self.menu = '''
1: Add member to the graph
2: Add relationship to the graph
3: Find the friends of a member
4: Find the shortest path between members
5: Find k new friends for a member
6: Remove member
7: Remove relationship
8: Exit
'''
    
    def run(self):
        while True:
            # Print menu
            self.display_graph()
            print(self.menu)
                
            # Input validation
            try:
                choice = int(input('Please enter a choice: '))
                if choice < 1 or choice > 8:
                    clear_output(wait=False)
                    print('Invalid input. Please enter a correct choice from 1 to 8')
                    continue
            except:
                clear_output(wait=False)
                print('Invalid input. Please enter an integer.')
                continue
                
            # Function selection 
            match choice:
                case 1:
                    name = input('Please enter the name of the member')
                    age = input('Please enter the age of the member')
                    if not age.isdigit():
                        clear_output(wait=False)
                        print('Invalid input. Please enter an integer for the age.')
                        continue
                    location = input('Please enter the location of the member')
                    self.add_member(name, age, location)
                    
                case 2:
                    member1 = input('Please enter the name of the member 1')
                    member2 = input('Please enter the name of the member 2')
                    self.add_relationship(member1, member2)
                    
                case 3:
                    member = input('Please enter the name of the member')
                    print(f'Friends: {self.find_friends(member)}')
                    
                case 4:
                    member1 = input('Please enter the name of the member 1')
                    member2 = input('Please enter the name of the member 2')
                    print(f'Shortest path: {self.shortest_path(member1, member2)}')
                    
                case 5:
                    member = input('Please enter the name of the member')
                    k = input('Please enter the number of the friends to be found')
                    if not k.isdigit():
                        clear_output(wait=False)
                        print('Invalid input. Please enter an integer for k.')
                        continue
                    print(self.find_k_new_friends(member, k))
                    
                case 6:
                    member = input('Please enter the name of the member')
                    self.remove_member(member)
                    
                case 7:
                    member1 = input('Please enter the name of the member 1')
                    member2 = input('Please enter the name of the member 2')
                    self.remove_relationship(member1, member2)
                    
                case 8:
                    clear_output(wait=False)
                    print('The program has been terminated')
                    break
                    
            input('Press any key to continue')
            clear_output(wait=False)

In [18]:
def sample_input(app):
    members = [
        ("Artem", 17, "Bluebell"),
        ("Tom", 17, "Bluebell"),
        ("Kevin", 18, "Rootes"),
        ("Femi", 18, "n/a"),
        ("Ruslan", 18, "Sherbourne"),
        ("Roma", 20, "Whitefields"),
        ("Alexander", 17, "Bluebell")
    ]

    for name, age, location in members:
        app.add_member(name, age, location)

    relationships = [
        ("Artem", "Tom"),
        ("Artem", "Kevin"),
        ("Tom", "Femi"),
        ("Tom", "Ruslan"),
        ("Tom", "Alexander"),
        ("Ruslan", "Roma")
    ]

    for name1, name2 in relationships:
        app.add_relationship(name1, name2)

In [19]:
app = GraphCLIApp()
sample_input(app)
app.run()

The program has been terminated
